In [4]:
# Import Dependencies
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine, MetaData, inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
from sqlalchemy.sql import text
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from flask import Flask, jsonify, render_template, request, redirect
import json
import pprint

In [5]:
# Create an engine for the database and connect
engine = create_engine("sqlite:///data.sqlite", echo=False)
engine.connect()

In [6]:
#Inspect table names
inspector = inspect(engine)
inspector.get_table_names()

['DPI', 'FPSR', 'PCE']

In [7]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine,reflect=True)
Base.classes.keys()

[]

In [8]:
# Create a reference to each table
class DPI(Base):
    __tablename__ = "DPI"
    __table_args__ = {"extend_existing":True}
    field1 = Column(Text,primary_key=True)

class FPSR(Base):
    __tablename__ = "FPSR"
    __table_args__ = {"extend_existing":True}
    DATE = Column(Text,primary_key=True)

class PCE(Base):
    __tablename__ = "PCE"
    __table_args__ = {"extend_existing":True}
    GeoName = Column(Text,primary_key=True)
    Line = Column(Text,primary_key=True)

Base.prepare()
session = Session(engine)
app = Flask(__name__)

In [9]:
GeoName_List = [g.GeoName for g in session.query(PCE.GeoName).distinct()]
GeoName_List

['United States',
 'Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'District of Columbia',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming',
 'New England',
 'Mideast',
 'Great Lakes',
 'Plains',
 'Southeast',
 'Southwest',
 'Rocky Mountain',
 'Far West']

In [10]:
Description_List = [d.Description for d in session.query(PCE.Description).distinct()]
Description_List

['Personal consumption expenditures',
 'Goods',
 'Durable goods',
 'Motor vehicles and parts',
 'Furnishings and durable household equipment',
 'Recreational goods and vehicles',
 'Other durable goods',
 'Nondurable goods',
 'Food and beverages purchased for off-premises consumption',
 'Clothing and footwear',
 'Gasoline and other energy goods',
 'Other nondurable goods',
 'Services',
 'Household consumption expenditures (for services)',
 'Housing and utilities',
 'Health care',
 'Transportation services',
 'Recreation services',
 'Food services and accommodations',
 'Financial services and insurance',
 'Other services',
 'Final consumption expenditures of nonprofit institutions serving households (NPISHs)',
 'Gross output of nonprofit institutions',
 'Less: Receipts from sales of goods and services by nonprofit institutions']

In [11]:
Component_Name = [c.ComponentName for c in session.query(PCE.ComponentName).distinct()]
Component_Name

['Total personal consumption expenditures (PCE) by state (millions of dollars)']

In [12]:
sql = "select * from PCE"
df = pd.read_sql(sql, engine)
df.drop(['GeoFIPS', 'ComponentId', 'Line', 'IndustryClassification'], axis=1, inplace=True)
df.head(10)

,GeoName,Region,ComponentName,Description,1997,1998,1999,2000,2001,2002,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,United States,,Total personal consumption expenditures (PCE) ...,Personal consumption expenditures,5557762,5899792,6303893,6789177,7099701,7380019,...,9745473,10008303,9840449,10196850,10682155,11044056,11355529,11858228,12327828,12816386
1,United States,,Total personal consumption expenditures (PCE) ...,Goods,2003806,2105478,2283640,2449635,2521834,2594547,...,3356541,3370357,3191925,3357470,3589317,3732508,3828877,3965012,4028728,4117111
2,United States,,Total personal consumption expenditures (PCE) ...,Durable goods,715530,779295,855570,912571,941515,985367,...,1184633,1102283,1023319,1070692,1125327,1191860,1241693,1296383,1367137,1411006
3,United States,,Total personal consumption expenditures (PCE) ...,Motor vehicles and parts,293083,320205,350730,363227,383289,401347,...,400573,339630,317058,341955,363458,395763,416132,441874,472169,480792
4,United States,,Total personal consumption expenditures (PCE) ...,Furnishings and durable household equipment,160491,173579,191185,208106,214940,225868,...,283458,268714,244311,250417,260672,271383,281557,294270,311516,325154
5,United States,,Total personal consumption expenditures (PCE) ...,Recreational goods and vehicles,174632,191360,210922,230858,234934,244772,...,335780,329331,303775,312735,320981,336605,348100,357404,372392,385474
6,United States,,Total personal consumption expenditures (PCE) ...,Other durable goods,87324,94151,102733,110380,108352,113380,...,164822,164608,158175,165585,180216,188109,195904,202835,211060,219586
7,United States,,Total personal consumption expenditures (PCE) ...,Nondurable goods,1288276,1326183,1428070,1537064,1580319,1609180,...,2171908,2268074,2168606,2286778,2463990,2540648,2587184,2668629,2661591,2706105
8,United States,,Total personal consumption expenditures (PCE) ...,Food and beverages purchased for off-premises ...,474776,487437,515530,540579,564003,575052,...,737331,772884,769958,788899,829146,848789,857547,884403,898957,915074
9,United States,,Total personal consumption expenditures (PCE) ...,Clothing and footwear,247496,257805,271097,280832,277868,278836,...,323692,319506,306498,320558,338946,354287,363625,375954,385461,393654


In [25]:
# Return each row as a dictionary and add it to a list
dict_list = []
for data in df.to_dict(orient='records'):
    dict_list.append(data)

dict_list

[{'1997': '5557762',
  '1998': '5899792',
  '1999': '6303893',
  '2000': '6789177',
  '2001': '7099701',
  '2002': '7380019',
  '2003': '7760899',
  '2004': '8255300',
  '2005': '8789586',
  '2006': '9299424',
  '2007': '9745473',
  '2008': '10008303',
  '2009': '9840449',
  '2010': '10196850',
  '2011': '10682155',
  '2012': '11044056',
  '2013': '11355529',
  '2014': '11858228',
  '2015': '12327828',
  '2016': '12816386',
  'ComponentName': 'Total personal consumption expenditures (PCE) by state (millions of dollars)',
  'Description': 'Personal consumption expenditures',
  'GeoName': 'United States',
  'Region': ''},
 {'1997': '2003806',
  '1998': '2105478',
  '1999': '2283640',
  '2000': '2449635',
  '2001': '2521834',
  '2002': '2594547',
  '2003': '2716926',
  '2004': '2895537',
  '2005': '3075755',
  '2006': '3231229',
  '2007': '3356541',
  '2008': '3370357',
  '2009': '3191925',
  '2010': '3357470',
  '2011': '3589317',
  '2012': '3732508',
  '2013': '3828877',
  '2014': '3965